# Importing Common Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import timedelta
from datetime import time
from datetime import datetime
import pickle

In [3]:
data=pd.read_csv('data/BOM500002.csv')
df=data
df=df.set_index('Date')
df.tail(5)

,Close
Date,
24-05-2021,1474.80
25-05-2021,1494.55
26-05-2021,1501.50
27-05-2021,1576.50
28-05-2021,1590.75


In [4]:
training_set=df.iloc[:,0:1].values
training_set

array([[ 650.  ],
       [ 600.  ],
       [ 550.  ],
       ...,
       [1501.5 ],
       [1576.5 ],
       [1590.75]])

In [5]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1),copy=True)
training_set_scaled=sc.fit_transform(training_set)
training_set_scaled

array([[0.10608387],
       [0.09527002],
       [0.08445617],
       ...,
       [0.29024374],
       [0.30646452],
       [0.30954647]])

### Creating a data structure with 60 timesteps and 1 output

In [6]:
import numpy as np
X_train=[]
y_train=[]
for i in range(60,len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)

### Reshaping

In [7]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## Part 2 - Building and Training the RNN

### Importing the Keras libraries and packages

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### Initialising the RNN

In [9]:
regressor = Sequential()

### Adding the first LSTM layer and some Dropout regularisation

In [10]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

### Adding a second LSTM layer and some Dropout regularisation

In [11]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a third LSTM layer and some Dropout regularisation

In [12]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a fourth LSTM layer and some Dropout regularisation

In [13]:
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

### Adding the output layer

In [14]:
regressor.add(Dense(units = 1))

### Compiling the RNN

In [15]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

### Fitting the RNN to the Training set

In [16]:
regressor.fit(X_train, y_train, epochs = 5, batch_size = 32)

Epoch 1/5
195/195 [==============================] - 43s 143ms/step - loss: 0.0073
Epoch 2/5
195/195 [==============================] - 28s 142ms/step - loss: 0.0017
Epoch 3/5
195/195 [==============================] - 28s 141ms/step - loss: 0.0014
Epoch 4/5
195/195 [==============================] - 27s 140ms/step - loss: 0.0013
Epoch 5/5
195/195 [==============================] - 28s 143ms/step - loss: 0.0012


In [17]:
regressor.save("BOM500002_model")

INFO:tensorflow:Assets written to: BOM500002_model\assets


INFO:tensorflow:Assets written to: BOM500002_model\assets
